# Evolution of the gas metal abundance

In [2]:
import numpy as np
from scipy.stats import binned_statistic
import pandas as pd
import warnings
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm

In [3]:
from auriga.snapshot import Snapshot
from auriga.settings import Settings
from auriga.paths import Paths
from auriga.parser import parse
from auriga.support import make_snapshot_number
from auriga.images import figure_setup
from auriga.mathematics import mad
from auriga.cosmology import Cosmology

In [4]:
figure_setup()

In [5]:
N_BINS_ABUNDANCE: int = 50
N_BINS_TIME: int = 50
MAX_SPHERICAL_RADIUS: float = 30.0

In [6]:
def read_data(simulation: str, abundances: list) -> tuple:
    """
    This method returns data of interest for this analysis.

    Parameters
    ----------
    simulation : str
        The simulation to consider.
    abundances : tuple
        A list of tuples with the abundances to calculate.

    Returns
    -------
    pd.DataFrame
        A data frame with the properties.
    """
    cosmology = Cosmology()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_extra_coordinates()

    for of, to in abundances:
        s.add_metal_abundance(of, to)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_gas = (s.type == 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)
    return_mask = (is_real_star | is_gas) & is_main_obj

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        data = {
            "ParticleType": s.type[return_mask],
            "FormationTime_Gyr": cosmology.expansion_factor_to_time(
                s.stellar_formation_time[return_mask]),
            "SphericalRadius_ckpc": s.r[return_mask],
        }

    for of, to in abundances:
        data[f"[{of}/{to}]"] = s.metal_abundance[f"{of}/{to}"][return_mask]
        data[f"[{of}/{to}]"][~np.isfinite(data[f"[{of}/{to}]"])] = np.nan

    data = pd.DataFrame(data)

    for of, to in abundances:
        data = data[data[f"[{of}/{to}]"].notna()]

    return data

In [7]:
def calculate_gas_data(simulation: str, abundances: list,
                       abundance_extents: list,
                       max_radius: float = np.inf) -> dict:
    settings = Settings()
    data = {}
    galaxy, rerun, resolution = parse(simulation)

    # Construct the bin edges for time and abundance
    rerun_txt = "re" if rerun else "or"
    data["TimeEdges_Gyr"] = pd.read_csv(
        f"../results/au{galaxy}_{rerun_txt}_l{resolution}/temporal_data.csv")[
            "Time_Gyr"].to_numpy()

    n_snapshots = make_snapshot_number(rerun, resolution)
    pdf_data = np.nan * np.ones((N_BINS_ABUNDANCE,
                                 data["TimeEdges_Gyr"].shape[0],
                                 len(abundances)))
    for i in range(n_snapshots):
        if i >= settings.first_snap:
            df = read_data(simulation + f"_s{i}", abundances)
            is_type = df["ParticleType"] == 0
            is_inner = df["SphericalRadius_ckpc"] <= max_radius
            for j, (of, to) in enumerate(abundances):
                pdf, bin_edges = np.histogram(
                    df[f"[{of}/{to}]"][is_type & is_inner], density=True,
                    range=(abundance_extents[j][0], abundance_extents[j][1]),
                    bins=N_BINS_ABUNDANCE)
                pdf_data[:, i, j] = pdf
                if f"[{of}/{to}]_AbundanceEdges" not in data.keys():
                    data[f"[{of}/{to}]_AbundanceEdges"] = bin_edges

    for j, (of, to) in enumerate(abundances):
        data[f"Gas_[{of}/{to}]_PDF"] = pdf_data[:, :, j]
    return data 

In [8]:
def calculate_stellar_data(simulation: str, abundances: list,
                           max_radius: float = np.inf) -> dict:
    settings = Settings()
    data = {}
    galaxy, rerun, resolution = parse(simulation)

    n_snapshots = make_snapshot_number(rerun, resolution)
    df = read_data(simulation + f"_s{n_snapshots - 1}", abundances)
    is_star = df["ParticleType"] == 4
    is_inner = df["SphericalRadius_ckpc"] <= max_radius
    for j, (of, to) in enumerate(abundances):
        abundance = df[f"[{of}/{to}]"][is_star & is_inner]
        medians, bin_edges, _ = binned_statistic(
            df["FormationTime_Gyr"][is_star & is_inner],
            abundance, statistic=np.median, bins=N_BINS_TIME, range=(0, 14))
        mads = binned_statistic(
            df["FormationTime_Gyr"][is_star & is_inner],
            abundance, statistic=mad, bins=N_BINS_TIME, range=(0, 14))[0]
        bin_centers = bin_edges[1:] - np.diff(bin_edges) / 2

        if "FormationTime_Gyr" not in data.keys():
            data["FormationTime_Gyr"] = bin_centers
        
        data[f"Star_[{of}/{to}]_Median"] = medians
        data[f"Star_[{of}/{to}]_MAD"] = mads

    return data

In [9]:
simulation = "au6_or_l4"
galaxy = parse(simulation)[0]
label = f"Au{galaxy}"

In [10]:
gas = calculate_gas_data(
    "au6_or_l4",
    [("Fe", "H"), ("O", "Fe")],
    [(-2.5, 1.5), (0, 1)],
    MAX_SPHERICAL_RADIUS)

In [11]:
stars = calculate_stellar_data(
    "au6_or_l4",
    [("Fe", "H"), ("O", "Fe")],
    MAX_SPHERICAL_RADIUS)

In [12]:
fig = plt.figure(figsize=(3.0, 2.5))
gs = fig.add_gridspec(nrows=1, ncols=1, hspace=0.0, wspace=0.0)
ax = gs.subplots(sharex=True, sharey=True)

ax.set_xlim(0, 14)
ax.set_xlabel("Time [Gyr]")
ax.set_xticks([0, 2, 4, 6, 8, 10, 12, 14])

ax.set_ylim(-2.5, 1.5)
ax.set_ylabel("[Fe/H]")

ax.tick_params(which='both', direction="in")
ax.label_outer()

im = ax.pcolormesh(
    gas["TimeEdges_Gyr"], gas["[Fe/H]_AbundanceEdges"][:-1],
    gas["Gas_[Fe/H]_PDF"],
    cmap="viridis", norm=LogNorm(0.001, vmax=5.0), rasterized=True)

cbar = fig.colorbar(im, ax=ax, label="PDF", ticks=[0.001, 0.01, 0.1, 1.0])
cbar.ax.set_yticklabels([0.001, 0.01, 0.1, 1.0])

ax.plot(stars["FormationTime_Gyr"][1:],
        stars["Star_[Fe/H]_Median"][1:], c='k', lw=0.75)
ax.plot(stars["FormationTime_Gyr"][1:],
        stars["Star_[Fe/H]_Median"][1:] - stars["Star_[Fe/H]_MAD"][1:],
        c='k', ls=(0, (5, 1)), lw=0.75)
ax.plot(stars["FormationTime_Gyr"][1:],
        stars["Star_[Fe/H]_Median"][1:] + stars["Star_[Fe/H]_MAD"][1:],
        c='k', ls=(0, (5, 1)), lw=0.75)
ax.plot(stars["FormationTime_Gyr"][1:],
        stars["Star_[Fe/H]_Median"][1:] - 2 * stars["Star_[Fe/H]_MAD"][1:],
        c='k', ls=(0, (1, 1)), lw=0.75)
ax.plot(stars["FormationTime_Gyr"][1:],
        stars["Star_[Fe/H]_Median"][1:] + 2 * stars["Star_[Fe/H]_MAD"][1:],
        c='k', ls=(0, (1, 1)), lw=0.75)

ax.text(x=0.05, y=0.95, size=8.0, s=r"$\texttt{" + label + r"}$",
        ha='left', va='top', transform=ax.transAxes)

fig.savefig(f"../images/gas_abundance/au6_FeH_abundance_evol.pdf")
plt.close(fig)

In [13]:
fig = plt.figure(figsize=(3.0, 2.5))
gs = fig.add_gridspec(nrows=1, ncols=1, hspace=0.0, wspace=0.0)
ax = gs.subplots(sharex=True, sharey=True)

ax.set_xlim(0, 14)
ax.set_xlabel("Time [Gyr]")
ax.set_xticks([0, 2, 4, 6, 8, 10, 12, 14])

ax.set_ylim(0, 1)
ax.set_ylabel("[O/Fe]")

ax.tick_params(which='both', direction="in")
ax.label_outer()

im = ax.pcolormesh(
    gas["TimeEdges_Gyr"], gas["[O/Fe]_AbundanceEdges"][:-1],
    gas["Gas_[O/Fe]_PDF"],
    cmap="viridis", norm=LogNorm(0.01, vmax=30.0), rasterized=True)

cbar = fig.colorbar(im, ax=ax, label="PDF", ticks=[0.01, 0.1, 1.0, 10])
cbar.ax.set_yticklabels([0.01, 0.1, 1.0, 10])

ax.plot(stars["FormationTime_Gyr"][1:],
        stars["Star_[O/Fe]_Median"][1:], c='k', lw=0.75)
ax.plot(stars["FormationTime_Gyr"][1:],
        stars["Star_[O/Fe]_Median"][1:] - stars["Star_[O/Fe]_MAD"][1:],
        c='k', ls=(0, (5, 1)), lw=0.75)
ax.plot(stars["FormationTime_Gyr"][1:],
        stars["Star_[O/Fe]_Median"][1:] + stars["Star_[O/Fe]_MAD"][1:],
        c='k', ls=(0, (5, 1)), lw=0.75)
ax.plot(stars["FormationTime_Gyr"][1:],
        stars["Star_[O/Fe]_Median"][1:] - 2 * stars["Star_[O/Fe]_MAD"][1:],
        c='k', ls=(0, (1, 1)), lw=0.75)
ax.plot(stars["FormationTime_Gyr"][1:],
        stars["Star_[O/Fe]_Median"][1:] + 2 * stars["Star_[O/Fe]_MAD"][1:],
        c='k', ls=(0, (1, 1)), lw=0.75)

ax.text(x=0.05, y=0.95, size=8.0, s=r"$\texttt{" + label + r"}$",
        ha='left', va='top', transform=ax.transAxes)

fig.savefig(f"../images/gas_abundance/au6_OFe_abundance_evol.pdf")
plt.close(fig)